In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./mnist_data/MNIST\raw\train-images-idx3-ubyte.gz to ./mnist_data/MNIST\raw




Extracting ./mnist_data/MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist_data/MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./mnist_data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST\raw




Extracting ./mnist_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST\raw

Processing...


C:\Users\megha\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [2]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

In [8]:
# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()

In [9]:
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [10]:
optimizer = optim.Adam(vae.parameters())
#kullback liebler divergence: 
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [14]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        #data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [15]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            #data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [16]:
for epoch in range(1, 51):
    train(epoch)
    test()

C:\Users\megha\anaconda3\lib\site-packages\torch\nn\functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Loss: 546.457930
Train Epoch: 1 [10000/60000 (17%)]	Loss: 192.215918
Train Epoch: 1 [20000/60000 (33%)]	Loss: 168.058242
Train Epoch: 1 [30000/60000 (50%)]	Loss: 165.294043
Train Epoch: 1 [40000/60000 (67%)]	Loss: 158.566318
Train Epoch: 1 [50000/60000 (83%)]	Loss: 164.200605
====> Epoch: 1 Average loss: 179.7384
====> Test set loss: 163.5262
Train Epoch: 2 [0/60000 (0%)]	Loss: 165.377910
Train Epoch: 2 [10000/60000 (17%)]	Loss: 156.236221
Train Epoch: 2 [20000/60000 (33%)]	Loss: 156.747285
Train Epoch: 2 [30000/60000 (50%)]	Loss: 152.943428
Train Epoch: 2 [40000/60000 (67%)]	Loss: 159.305098
Train Epoch: 2 [50000/60000 (83%)]	Loss: 150.348818
====> Epoch: 2 Average loss: 158.4346
====> Test set loss: 154.9907
Train Epoch: 3 [0/60000 (0%)]	Loss: 152.372285
Train Epoch: 3 [10000/60000 (17%)]	Loss: 150.205566
Train Epoch: 3 [20000/60000 (33%)]	Loss: 150.766152
Train Epoch: 3 [30000/60000 (50%)]	Loss: 145.502148
Train Epoch: 3 [40000/60000 (67%)]	Loss: 158.10

Train Epoch: 22 [50000/60000 (83%)]	Loss: 145.693975
====> Epoch: 22 Average loss: 137.9601
====> Test set loss: 140.1020
Train Epoch: 23 [0/60000 (0%)]	Loss: 139.305498
Train Epoch: 23 [10000/60000 (17%)]	Loss: 143.239512
Train Epoch: 23 [20000/60000 (33%)]	Loss: 138.436777
Train Epoch: 23 [30000/60000 (50%)]	Loss: 135.039385
Train Epoch: 23 [40000/60000 (67%)]	Loss: 135.592334
Train Epoch: 23 [50000/60000 (83%)]	Loss: 134.140254
====> Epoch: 23 Average loss: 137.9480
====> Test set loss: 140.0510
Train Epoch: 24 [0/60000 (0%)]	Loss: 137.378457
Train Epoch: 24 [10000/60000 (17%)]	Loss: 137.692803
Train Epoch: 24 [20000/60000 (33%)]	Loss: 139.527256
Train Epoch: 24 [30000/60000 (50%)]	Loss: 138.810117
Train Epoch: 24 [40000/60000 (67%)]	Loss: 132.932637
Train Epoch: 24 [50000/60000 (83%)]	Loss: 143.631240
====> Epoch: 24 Average loss: 137.6502
====> Test set loss: 140.1291
Train Epoch: 25 [0/60000 (0%)]	Loss: 132.844619
Train Epoch: 25 [10000/60000 (17%)]	Loss: 143.815762
Train Epoch: 

KeyboardInterrupt: 